In [1]:
import pandas as pd
from common.data import DataLoader
from common.metrics import map_at_k
from models.popular import SegmentRecommender, PopularRecommender
from models.lightfm import WeightFeaturedLightFM
from recsys_course.const import *

/Users/d-emeljanov/pet_projects/recsys-course/notebooks/venv/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = DataLoader.from_folder(
    '../data/preprocessed/',
    user_col='user_id',
    item_col='item_id',
    date_col='last_watch_dt',
    watched_pct_min=0
)

In [3]:
train = data.interactions.copy()
train[DATE_COL] = pd.to_datetime(train[DATE_COL])

In [4]:
test = pd.read_csv('../data/raw/sample_submission.csv').drop('item_id', axis=1)
test

,user_id
0,3
1,11
2,29
3,30
4,33
...,...
193108,1097527
193109,1097537
193110,1097538
193111,1097544


In [19]:
cold_user_ids = set(test['user_id'].unique().tolist()).difference(set(train['user_id'].unique().tolist()))

test_cold = test[test['user_id'].isin(cold_user_ids)].reset_index(drop=True)
test_warm = test[~test['user_id'].isin(cold_user_ids)].reset_index(drop=True)

## Cold Predictions

In [20]:
test_cold_no_features = test_cold.loc[~test_cold['user_id'].isin(data.users['user_id'].tolist())].reset_index(drop=True)
test_cold_features = test_cold.loc[test_cold['user_id'].isin(data.users['user_id'].tolist())].reset_index(drop=True)

In [21]:
fallback = PopularRecommender(
    fb__min_watched_pct=10,
    fb__total_dur_min=2000,
    days=10,
    date_col=DATE_COL,
    user_col=USER_COL,
    item_col=ITEM_COL,
)

fallback.fit(train)

test_cold_no_features['item_id'] = fallback.recommend(test_cold_no_features['user_id'].tolist(), 10)

In [22]:
popular = SegmentRecommender(
    fb__min_watched_pct=10,
    fb__total_dur_min=2000,
    days=10,
    date_col=DATE_COL,
    user_col=USER_COL,
    item_col=ITEM_COL,
    segment=['age', 'sex']
)

popular.add_user_features(data.users)

popular.fit(train)

test_cold_features['item_id'] = popular.recommend(test_cold_features['user_id'].tolist(), 10)

32it [00:07,  4.03it/s]


In [23]:
test_cold_features

,user_id,item_id
0,70,"[15297, 9728, 10440, 4151, 3734, 13865, 2657, ..."
1,85,"[15297, 10440, 9728, 13865, 512, 3734, 12192, ..."
2,97,"[15297, 10440, 9728, 13865, 12192, 3734, 512, ..."
3,124,"[15297, 10440, 9728, 13865, 12192, 3734, 512, ..."
4,135,"[9728, 10440, 15297, 13865, 3734, 14488, 4151,..."
...,...,...
46203,1097453,"[15297, 10440, 9728, 3734, 4151, 13865, 12192,..."
46204,1097494,"[9728, 10440, 15297, 13865, 3734, 14488, 4151,..."
46205,1097537,"[9728, 10440, 13865, 15297, 512, 12192, 14488,..."
46206,1097538,"[9728, 13865, 10440, 15297, 3734, 512, 4685, 1..."


## Warm Predictions

In [40]:
lfm = WeightFeaturedLightFM(
    notseen_watched_upper=95,
    notseen_watched_lower=5,
    no_components=150,
    date_col=DATE_COL,
    user_col=USER_COL,
    item_col=ITEM_COL,
    user_features_col=None,
    item_features_col=None,
    preprocess_array_split=None
)

lfm.fit(train)

test_warm['recs'] = lfm.recommend(test_warm['user_id'].tolist(), 200)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128170/128170 [11:12<00:00, 190.58it/s]


In [25]:
cbc_df = train.loc[:, ['user_id', 'item_id']]
cbc_df.loc[:, 'y'] = 1
cbc_unused = data.unused.copy()
cbc_unused['y'] = 0

cbc_df = pd.merge(
    left=pd.merge(
        left=pd.concat([cbc_df, cbc_unused]),
        right=data.users,
        on=[USER_COL],
        how='left'
    ),
    right=data.items,
    on=[ITEM_COL],
    how='left'
)

cbc_df

,user_id,item_id,y,age,income,sex,kids_flg,content_type,title,title_orig,...,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat,for_kids_rating
0,176549,9506,1,age_35_44,income_40_60,m,0.0,film,холодное сердце,Frozen,...,сша,-1,0.0,unknown,"крис бак, дженнифер ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн...",2010-2020,1
1,699317,1659,1,age_35_44,income_40_60,m,0.0,film,три богатыря. ход конем,Tri bogatyrya. Khod konem,...,россия,-1,6.0,unknown,к. феоктистов,"Сергей Маковецкий, Дмитрий Высоцкий, Дмитрий Н...",Придворный конь Гай Юлий Цезарь на свою беду п...,"2014, россия, три, богатыря, ход, конем",2010-2020,1
2,656683,7107,1,age_25_34,income_60_90,m,0.0,series,девятаев,V2. Escape from Hell,...,россия,-1,12.0,unknown,тимур бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,Unknown,2020_inf,1
3,864613,7638,1,age_65_inf,income_20_40,zh,0.0,series,мишель,Mishel',...,россия,-1,12.0,unknown,ольга перуновская,"Арина Постникова, Алексей Демидов, Евгения Нох...",Мишель получила свое необычное имя благодаря о...,Unknown,2010-2020,1
4,964868,9506,1,age_25_34,income_20_40,zh,0.0,film,холодное сердце,Frozen,...,сша,-1,0.0,unknown,"крис бак, дженнифер ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн...",2010-2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15098036,1097557,12597,0,age_35_44,income_20_40,zh,0.0,film,эволюция борна,The Bourne Legacy,...,"сша, япония",-1,16.0,unknown,тони гилрой,"Джереми Реннер, Джоан Аллен, Донна Мерфи, Дэви...",Параллельно с программой ЦРУ по подготовке кил...,"убийца, волк, сеул, мэриленд, тайная операция,...",2010-2020,0
15098037,1097557,9332,0,age_35_44,income_20_40,zh,0.0,film,заноза,Splinter,...,сша,-1,18.0,unknown,тоби уилкинс,"Джилл Вагнер, Лорел Уитсетт, Паоло Костанзо, Р...",Молодая пара оказывается заперта на заброшенно...,"Заноза, 2008, США, притяжение, противоположнос...",2000-2010,0
15098038,1097557,11352,0,age_35_44,income_20_40,zh,0.0,film,тёмная сторона,Virtualia Episode Three: Dark Side,...,швеция,-1,21.0,unknown,антонио адамо,"Линн Стоун, Клаудия Риччи, Софи Эванс, Сандра ...",Приключенческая эротика в духе Индианы Джонса ...,", Анальный секс, Банкомат, Взгляд в камеру, Вт...",2000-2010,0
15098039,1097557,15512,0,age_35_44,income_20_40,zh,0.0,series,в последний раз прощаюсь,None,...,украина,-1,16.0,unknown,андрей черных,"Александра Польгуй, Андрей Романий, Андрей Фед...",Когда-то в молодости Андрей спас тонущую девоч...,"последний, раз, прощаюсь, 2017, Украина, безот...",2010-2020,0


In [42]:
cbc_df['sex'] = cbc_df['sex'].fillna('unknown')
cbc_df['age'] = cbc_df['age'].fillna('age_unknown')
cbc_df['genres'] = cbc_df['genres'].fillna('genres_unknown')
cbc_df['income'] = cbc_df['income'].fillna('income_unknown')


In [50]:
cbc_df['keywords'].map(lambda x: x.lower().replace(', '))

0           королева, мюзикл, принцесса, предательство, сн...
1                     2014, россия, три, богатыря, ход, конем
2                                                     Unknown
3                                                     Unknown
4           королева, мюзикл, принцесса, предательство, сн...
                                  ...                        
15098036    убийца, волк, сеул, мэриленд, тайная операция,...
15098037    Заноза, 2008, США, притяжение, противоположнос...
15098038    , Анальный секс, Банкомат, Взгляд в камеру, Вт...
15098039    последний, раз, прощаюсь, 2017, Украина, безот...
15098040    Крепость, колесах, 1960, СССР, армия, Великая,...
Name: keywords, Length: 15098041, dtype: object

In [43]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(
    cat_features=['sex', 'age', 'income', 'release_year_cat', 'content_type'],
    text_features=['genres', 'keywords'],
    iterations=100
)

cbc.fit(cbc_df, cbc_df['y'])

Learning rate set to 0.5
0:	learn: 0.6535461	total: 3.67s	remaining: 6m 3s
1:	learn: 0.5751750	total: 7.89s	remaining: 6m 26s
2:	learn: 0.5177812	total: 11.5s	remaining: 6m 12s
3:	learn: 0.4907583	total: 15.6s	remaining: 6m 13s
4:	learn: 0.4704460	total: 19.6s	remaining: 6m 12s
5:	learn: 0.4661009	total: 23.2s	remaining: 6m 3s
6:	learn: 0.4562473	total: 26.6s	remaining: 5m 53s
7:	learn: 0.4524435	total: 30.2s	remaining: 5m 47s
8:	learn: 0.4480730	total: 33s	remaining: 5m 33s
9:	learn: 0.4414583	total: 36.6s	remaining: 5m 29s
10:	learn: 0.4391950	total: 40.1s	remaining: 5m 24s
11:	learn: 0.4361192	total: 43.3s	remaining: 5m 17s
12:	learn: 0.4330321	total: 46.6s	remaining: 5m 11s
13:	learn: 0.4315821	total: 49.3s	remaining: 5m 3s
14:	learn: 0.4297928	total: 52s	remaining: 4m 54s
15:	learn: 0.4282261	total: 55s	remaining: 4m 48s
16:	learn: 0.4276638	total: 57.7s	remaining: 4m 41s
17:	learn: 0.4269812	total: 1m	remaining: 4m 37s
18:	learn: 0.4252310	total: 1m 3s	remaining: 4m 31s
19:	learn

In [44]:
test_pred = pd.merge(
    left=pd.merge(
        left=test_warm.explode('recs').rename(columns={'recs': 'item_id'}),
        right=data.items,
        on=['item_id'],
        how='left'
    ),
    right=data.users,
    on=['user_id'],
    how='left'
)

test_pred['sex'] = test_pred['sex'].fillna('unknown')
test_pred['income'] = test_pred['income'].fillna('income_unknown')
test_pred['age'] = test_pred['age'].fillna('age_unknown')
test_pred['genres'] = test_pred['genres'].fillna('genres_unknown')

test_pred['rating'] = cbc.predict_proba(test_pred[['sex', 'age', 'income', 'release_year_cat', 'content_type', 'genres']])[:, 1]
test_pred = (
    test_pred
    .groupby('user_id')
    .apply(lambda x: x.sort_values('rating', ascending=False)['item_id'].tolist()[:10])
)

In [45]:
(
    test.shape[0]
) == (
    test_cold_no_features.shape[0]
    + test_cold_features.shape[0]
    + test_pred.shape[0]
)

True

In [46]:
test_cold_no_features.isna().sum()

user_id    0
item_id    0
dtype: int64

In [47]:
test_cold_features.isna().sum()

user_id    0
item_id    0
dtype: int64

In [48]:
test_pred.reset_index().isna().sum()

user_id    0
0          0
dtype: int64

In [49]:
pd.concat([
    test_cold_no_features,
    test_cold_features,
    test_pred.reset_index().rename(columns={0: 'item_id'})
]).to_csv('../data/submit.csv', index=None)

,user_id,item_id
0,3,"[11863, 7107, 13865, 15297, 7829, 10440, 3935,..."
1,11,"[13865, 7107, 7829, 15399, 12995, 12981, 3935,..."
2,30,"[7107, 13865, 15297, 7829, 10440, 3935, 12981,..."
3,39,"[7107, 11863, 13865, 7829, 10440, 3935, 12995,..."
4,46,"[13865, 7107, 7829, 15297, 15399, 12981, 12995..."
...,...,...
128165,1097513,"[13865, 15297, 12981, 12995, 10440, 9728, 8447..."
128166,1097521,"[13865, 11863, 7107, 7829, 15297, 12981, 12995..."
128167,1097525,"[13865, 7829, 15297, 12995, 12981, 10440, 9728..."
128168,1097527,"[7107, 11863, 13865, 15297, 7829, 10440, 3935,..."
